In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Documents\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Documents\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
import dagshub
dagshub.init(repo_owner='nirmalkumarmaradiya', repo_name='Kidney-Disease-Classification-Deep_Learning-Project', mlflow=True)



Initialized MLflow to track repo "nirmalkumarmaradiya/Kidney-Disease-Classification-Deep_Learning-Project"

Repository nirmalkumarmaradiya/Kidney-Disease-Classification-Deep_Learning-Project initialized!

In [6]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")



In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/nirmalkumarmaradiya/Kidney-Disease-Classification-Deep_Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
import mlflow
from urllib.parse import urlparse
from pathlib import Path
import tensorflow as tf
import json
import psutil

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

        # Enable memory growth for GPUs
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                print("Enabled memory growth for GPUs")
            except RuntimeError as e:
                print(f"Error enabling memory growth for GPUs: {e}")

    def _valid_generator(self):
        try:
            datagenerator_kwargs = dict(
                rescale=1./255,
                validation_split=0.30
            )

            dataflow_kwargs = dict(
                target_size=self.config.params_image_size[:-1],
                batch_size=2,  # Further reduced batch size
                interpolation="bilinear"
            )

            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset="validation",
                shuffle=False,
                **dataflow_kwargs
            )
            print("Validation generator set up successfully.")
        except Exception as e:
            print(f"An error occurred while setting up validation generator: {e}")
            raise e

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        try:
            return tf.keras.models.load_model(path)
        except Exception as e:
            print(f"An error occurred while loading the model: {e}")
            raise e

    def evaluate(self):
        try:
            print("Loading model...")
            self.model = self.load_model(self.config.path_of_model)
            print("Model loaded successfully.")

            print("Setting up validation generator...")
            self._valid_generator()
            print("Validation generator set up successfully.")

            print("Evaluating model...")

            # Track memory usage before evaluation
            mem_before = psutil.virtual_memory().available
            print(f"Available memory before evaluation: {mem_before} bytes")

            # Evaluate model
            self.score = self.model.evaluate(self.valid_generator)

            # Track memory usage after evaluation
            mem_after = psutil.virtual_memory().available
            print(f"Available memory after evaluation: {mem_after} bytes")

            print(f"Model evaluated successfully. Score: {self.score}")

            self.save_score()
            print("Scores saved successfully.")
        except Exception as e:
            print(f"An error occurred during evaluation: {e}")
            raise e
        finally:
            # Clear session to free memory
            tf.keras.backend.clear_session()
            print("Session cleared.")

    def save_score(self):
        try:
            scores = {"loss": self.score[0], "accuracy": self.score[1]}
            with open("scores.json", "w") as f:
                json.dump(scores, f)
            print("Scores saved to scores.json.")
        except Exception as e:
            print(f"An error occurred while saving scores: {e}")
            raise e

    def log_into_mlflow(self):
        try:
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
            print("Logged into MLflow successfully.")
        except Exception as e:
            print(f"An error occurred while logging into MLflow: {e}")
            raise e

# Example usage within a try-except block
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()

    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    print(f"An error occurred: {e}")
    raise e


[2024-07-02 11:16:29,963: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-02 11:16:29,987: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-02 11:16:29,996: INFO: common: created directory at: artifacts]
Loading model...
Model loaded successfully.
Setting up validation generator...
Found 139 images belonging to 2 classes.
Validation generator set up successfully.
Validation generator set up successfully.
Evaluating model...
Available memory before evaluation: 1139232768 bytes
70/70 [==============================] - 65s 872ms/step - loss: 0.0273 - accuracy: 0.9856
Available memory after evaluation: 1099808768 bytes
Model evaluated successfully. Score: [0.02734319493174553, 0.9856114983558655]
Scores saved to scores.json.
Scores saved successfully.
Session cleared.


2024/07/02 11:17:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-02 11:17:51,190: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpkalrb15v\model\data\model\assets
[2024-07-02 11:17:57,153: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpkalrb15v\model\data\model\assets]


2024/07/02 11:20:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\DELL\AppData\Local\Temp\tmpkalrb15v\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\DELL\.conda\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/07/02 11:20:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


Logged into MLflow successfully.
